In [10]:
import pickle
import pandas as pd
import torch

import python.voxels as voxels
import python.RNN as RNN
import python.clustering as cl

project_folder = "veleval"

In [11]:
with open("./files/"+project_folder+"/data_processed/osmnx_pathfinding_simplified.df",'rb') as infile:
    df_pathfinding = pickle.load(infile)
with open("./files/"+project_folder+"/clustering/dbscan_observations.tab",'rb') as infile:
    tab_clusters = pickle.load(infile)
with open("./files/"+project_folder+"/clustering/voxels_clustered_osmnx.dict",'rb') as infile:
    dict_voxels = pickle.load(infile)
with open("./files/"+project_folder+"/neural_networks/saved/network.param",'rb') as infile:
    param = pickle.load(infile)
with open("./files/"+project_folder+"/neural_networks/saved/num_test.tab",'rb') as infile:
    tab_num_test = pickle.load(infile)
    

size_data = 1

df = df_pathfinding

tab_routes_voxels, _, _ = voxels.generate_voxels(df, df.iloc[0]["route_num"], df.iloc[-1]["route_num"])
tab_routes_voxels_int = []

df_voxels = pd.DataFrame()
df_voxels_train = pd.DataFrame()

dict_clusters = cl.tab_clusters_to_dict(tab_clusters)

network = RNN.RNN_LSTM(size_data, max(tab_clusters)+1, param.hidden_size, param.num_layers, param.bidirectional, param.dropout)
network.load_state_dict(torch.load("files/"+project_folder+"/neural_networks/saved/network_temp.pt"))
network.eval()


RNN_LSTM(
  (lstm): LSTM(1, 256, num_layers=2, dropout=0.1, bidirectional=True)
  (out1): Linear(in_features=512, out_features=128, bias=True)
  (out2): Linear(in_features=128, out_features=74, bias=True)
  (softmax): LogSoftmax(dim=1)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [12]:
print(len(tab_num_test))

175


In [13]:
tab_results = []
tab_routes_voxels_int = []
for i in range(len(tab_routes_voxels)):
    nb_vox = 0
    tab_routes_voxels_int.append([])
    route = tab_routes_voxels[i]
    for vox in route:
        if(nb_vox%param.voxels_frequency==0): #(len(tab_routes_voxels_int[i])==0 or tab_routes_voxels_int[i][-1][0] != dict_voxels[vox]["cluster"]): 
            points = [dict_voxels[vox]["cluster"]+1]
            tab_routes_voxels_int[i].append(points)
        nb_vox += 1

In [14]:
print(tab_routes_voxels_int[0])

[[696], [696], [696], [794], [696], [823], [546], [546], [68], [68], [68], [68], [68], [693], [693], [693], [597], [46], [266], [46], [681], [353], [353], [1022], [1022], [1022], [120], [635], [3], [3], [741], [741], [741], [882], [882], [271], [977], [271], [39], [39], [607], [39], [607], [802], [182], [182], [537], [182], [182], [182], [182], [898]]


In [15]:
nb_prediction_test = 0
nb_good_prediction_test = 0
nb_prediction_train = 0
nb_good_prediction_train = 0
tab_fail_test = []
tab_fail_train = []

for i in range(len(tab_routes_voxels_int)):

    tens_route = torch.Tensor(tab_routes_voxels_int[i]).unsqueeze(1)

    hidden = network.initHidden()
    for j in range(tens_route.shape[0]):
        input = tens_route[j].unsqueeze(1)
        output, hidden = network(input, hidden)
    pred = output.argmax(dim=1, keepdim=True)
    
    tab_results.append({"probabilities": output, "good_result": pred==tab_clusters[i]})
    
    if(i in tab_num_test):
        tab_results[-1]["test"] = True
        nb_prediction_test+=1
        if(pred==tab_clusters[i]):
            nb_good_prediction_test+=1
    else:
        tab_results[-1]["test"] = False
        nb_prediction_train+=1
        if(pred==tab_clusters[i]):
            nb_good_prediction_train+=1
    
            
print("Good train predict:", nb_good_prediction_train/nb_prediction_train*100, "%")
print("Good test predict:", nb_good_prediction_test/nb_prediction_test*100, "%")

Good train predict: 82.04419889502762 %
Good test predict: 64.57142857142857 %


In [7]:
print(len(tab_routes_voxels_int), len(tab_routes_voxels))

899 899
